<a href="https://colab.research.google.com/github/PiotrekTrzeciak/Uczenie_Maszynowe_2025/blob/main/docs/DNN-Lab-4-MNIST-in-Pytorch-student-version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

In this exercise, we are using high-level abstractions from torch.nn like nn.Linear.
Note: during the next lab session we will go one level deeper and implement more things
with bare hands.

Tasks:

 1. Read the code.

 2. Check that the given implementation reaches 95% test accuracy for architecture input-128-128-10 after few epochs.

 3. Add the option to use SGD with momentum instead of ADAM.

 4. Experiment with different learning rates. Use the provided TrainingVisualizer
 to plot the learning curves and gradient-to-weight ratios. Compare visualizations
 for different learning rates for both ADAM and SGD with momentum.

 5. Parameterize the constructor by a list of sizes of hidden layers of the MLP.
 Note that this requires creating a list of layers as an attribute of the Net class,
 and one can't use a standard Python list containing nn.Modules (why?).
 Check torch.nn.ModuleList.

If you run this notebook locally then you may need to install some packages.
It may be achieved by adding the following code cell to the notebook and running it:
```
!pip install torch torchvision plotly ipywidgets
```
This notebook can also utilize Colab GPU. However, remember to kill your GPU session after classes as otherwise, you may use all your free GPU time for this week.

In [1]:
import math
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import plotly.graph_objects as go
from torch import Tensor
from torchvision import datasets, transforms

# Allow custom (non-ipywidget) widgets.
if "google.colab" in sys.modules:
    from google.colab import output

    output.enable_custom_widget_manager()

# For reproducibility.
torch.manual_seed(1)

## Tensors
In ML, *tensor* is just a short name for an n-dimensional array.

In other areas like physics, it might mean a multilinear operation that can be represented, after a choice of basis, by an ndarray; this is where we would see all the extra stuff (like covariance/contravariance), like for example on the Wikipedia article for "tensor". For us, it's usually just a bunch of numbers.

torch.Tensor is can be thought of as a wrapper around numpy ndarray, but:
* the interface is slightly different (like `np.expand_dims(x, dim=n-1)` vs `torch.unsqueeze(x, axis=-1)`);
* the data can be stored in GPU memory (or others);
* it can be part of a computation graph: a computed Tensor has pointers to other Tensors it was computed from, and to functions computing partial derivatives.

All the operations defined on for a torch.Tensor (`@`, `.arctan`, `.unsqueeze`, and a hundred others) can also run on a GPU and

In [2]:
x_np = np.array([[0, 1, 2], [3, 4, 5]])
x = torch.tensor([[0, 1, 2], [3, 4, 5]])  # Also: torch.rand, ones, zeros, rand_like, ...
display(x_np, x)

array([[0, 1, 2],
       [3, 4, 5]])

tensor([[0, 1, 2],
        [3, 4, 5]])

In [3]:
torch.from_numpy(x_np).equal(x), np.array_equal(x_np, x.numpy())

(True, True)

In [4]:
x.shape, x.dtype, x.device, x.requires_grad

(torch.Size([2, 3]), torch.int64, device(type='cpu'), False)

Most operations have an in-place variant with the same name, but with an underscore.
This forgets the original value, making the computation of gradients impossible, so most of the time we don't use in-place operations.

In [5]:
x.add_(2)
display(x)

tensor([[2, 3, 4],
        [5, 6, 7]])

## Autograd
Operations on torch tensors remember the computation graph.
This allows to compute the gradient of e.g. a loss node over all other nodes with loss.backward().

In [6]:
x = torch.tensor([[0., 1., 2.], [3., 4., 5.]], requires_grad=True)
y = torch.sigmoid(x)
loss = (((y - 1) ** 2).mean())
print("x", x.grad_fn, ", y", y.grad_fn, ", loss", loss.grad_fn)
print(loss.grad_fn.next_functions)  # type: ignore
print(loss.grad_fn.next_functions[0][0].next_functions)  # type: ignore

x None , y <SigmoidBackward0 object at 0x7f6d2de30a00> , loss <MeanBackward0 object at 0x7f6d2de33be0>
((<PowBackward0 object at 0x7f6d2de30a00>, 0),)
((<SubBackward0 object at 0x7f6d2de30a00>, 0),)


The method Tensor.backward():
* for each node with `requires_grad`: computes the gradients from each .grad_fn and propagates it using the chain rule.
* for each leaf node (`requires_grad` with no predecessors that `requires_grad`): accumulates the gradient into the tensor’s `.grad` attribute.



In [7]:
loss.backward()
x.grad

tensor([[-4.1667e-02, -1.7626e-02, -4.1719e-03],
        [-7.1418e-04, -1.0590e-04, -1.4831e-05]])

We say *accumulates* not *stores*, because if .grad already exists, `backward()` adds to it, instead of replacing it.

In [8]:
loss2 = ((x * 0).mean())
loss2.backward()
x.grad

tensor([[-4.1667e-02, -1.7626e-02, -4.1719e-03],
        [-7.1418e-04, -1.0590e-04, -1.4831e-05]])

During (for example) evaluation, you don't need to store the computational graph, so you can disable it:

In [9]:
with torch.no_grad():
    y = x * x

print(y.grad_fn)

None


In some situations you may need more fine-grained control.
You can detach a tensor from the graph, to treat it like a constant tensor.

In [10]:
y = x * x
print(y.grad_fn, y.requires_grad)
y = y.detach()  # Not in-place, returns a new tensor with the same data.
print(y.grad_fn, y.requires_grad)

<MulBackward0 object at 0x7f6d2ddca980> True
None False


## Modules
A torch **module** (not *model*) is essentially just an object with a `forward()` method, usually `forward(x: Tensor) -> Tensor`.
A network (model) is built as a large module that contains and calls smaller modules, which themselves contain even smaller modules.

Basically any callable could be a module, but subclassing `nn.Module` adds features like:
* automatically registering and enumerating submodules and trainable parameters,
* serializing and storing parameters into files,
* adding hooks in the middle of existing models.

In [11]:
class MyLinear(nn.Module):
    """
    This is essentially the same as nn.Linear.

    Compare: https://github.com/pytorch/pytorch/blob/v2.9.0/torch/nn/modules/linear.py#L53
    """
    def __init__(self, in_features: int, out_features: int) -> None:
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.weight = nn.Parameter(torch.empty((out_features, in_features)))
        self.bias = nn.Parameter(torch.empty(out_features))

        self.reset_parameters()

    def reset_parameters(self) -> None:
        # The initialization used by PyTorch by default, including non-zero biases.
        bound = 1 / math.sqrt(self.in_features)
        nn.init.uniform_(self.weight, -bound, bound)
        nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        """
        Input shape: (B*, in_features).
        Output shape: (B*, out_features).
        """
        return input @ self.weight.T + self.bias


We define the module behaviour in the **`forward()`** method, not in the special `__call__()`.<br>
However, we then use the module directly as a callable, `module()`.<br>
This will call `module.forward()`, but can also calls hooks, if any are defined.

In [12]:
module = MyLinear(2, 3)
x = torch.rand((7, 5, 2))
y = module(x)
y.shape

torch.Size([7, 5, 3])

The class **nn.Parameter** just wraps a tensor to indicate that this is a trainable parameter, and changes the default `requires_grad`.<br>
When an nn.Parameter is assigned to a field of an nn.Module, it automatically registers it,
so that all parameters of this modules and its submodules can be easily accessed.

In [13]:
for i, (name, param) in enumerate(module.named_parameters()):
    print(f'{i}. "{name}":\t{param}\n')

0. "weight":	Parameter containing:
tensor([[ 0.3643, -0.3121],
        [-0.1371,  0.3319],
        [-0.6657,  0.4241]], requires_grad=True)

1. "bias":	Parameter containing:
tensor([-0.1455,  0.3597,  0.0983], requires_grad=True)



## A basic Net

In [14]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x: Tensor) -> Tensor:
        """
        Input: shape (B, 28, 28).
        Output: class probabilities (after softmax), shape (B, 10).
        """
        x = torch.flatten(x, start_dim=1)  # (B, 728)
        x = self.fc1(x)  # (B, 128)
        x = F.relu(x)
        x = self.fc2(x)  # (B, 128)
        x = F.relu(x)
        x = self.fc3(x)  # (B, 10)
        output = F.log_softmax(x, dim=1)
        return output

Assigning to a nn.Module to a field of nn.Module automatically registers it as a submodule.<br>
You can iterate over all submodules (recursively) with `.named_modules()` or over direct children only with `.named_children()`.

In [15]:
net = Net()
for i, (name, submodule) in enumerate(net.named_modules()):
    print(f'{i}. "{name}":\t{submodule}')

0. "":	Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)
1. "fc1":	Linear(in_features=784, out_features=128, bias=True)
2. "fc2":	Linear(in_features=128, out_features=128, bias=True)
3. "fc3":	Linear(in_features=128, out_features=10, bias=True)


The `.train()` and `.eval()` methods set the `training` flag of a module and all its submodules.<br>
This changes the behaviour of certain modules like BatchNorm and Dropout (see next labs).

Forgetting to switch the training mode is a common bug that can cause weird behaviour.

In [16]:
net.train()
print(net.fc1.training)
net.eval()
print(net.fc1.training)

True
False


Note that operations on a module (`.train()`, `.eval()`, `.to(dtype)`, `.to(device)`) modify it in-place.<br>
Operations on a tensor without a trailing underscore (including `.to(dtype)`, `.to(device)`) return a new tensor and leave the original unchanged


## Train and test code

In [17]:
# @title <small>Helper class: TrainingVisualizer</small>

class TrainingVisualizer:
    def __init__(self, log_interval: int = 10):
        self.log_interval = log_interval
        self.train_loss_fig = self.init_line_plot(
            title="Training loss", xaxis_title="Step"
        )
        self.grad_to_weight_fig = self.init_line_plot(
            title="Gradient standard deviation to weight standard deviation ratio at 1st layer",
            xaxis_title="Step",
            yaxis_title="Gradient to weight ratio (log scale)",
            yaxis_type="log",
        )
        self.test_acc_fig = self.init_line_plot(
            title="Test accuracy", x=[], xaxis_title="Epoch", mode="lines+markers"
        )

        # Parameters related to current tracked model and its training
        self.first_linear_layer = None
        self.lr = None
        self.trace_idx = -1

    def init_line_plot(
        self,
        title: str,
        x=None,
        xaxis_title: str | None = None,
        yaxis_title: str | None = None,
        yaxis_type: str = "linear",
        mode: str = "lines",
    ) -> go.FigureWidget:
        fig = go.Figure()
        fig.update_layout(
            title=title,
            title_x=0.5,
            xaxis_title=xaxis_title,
            yaxis_title=yaxis_title,
            height=400,
            width=1500,
            margin=dict(b=10, t=60),
        )
        fig.update_yaxes(type=yaxis_type)
        # We cannot add new traces dynamically because Colab has a problem with widgets
        # from plotly (traces added dynamically are rendered twice).
        # As an ugly workaround we create a lot of empty traces and update them later
        # with actual data. Empty traces are not plotted.
        for _ in range(25):
            fig.add_trace(go.Scatter(x=x, y=[], showlegend=True, mode=mode))

        fig_widget = go.FigureWidget(fig)
        display(fig_widget)
        return fig_widget

    def track_model(
        self, model: torch.nn.Module, optimizer: torch.optim.Optimizer, lr: float
    ) -> None:
        """
        Start tracking training metrics for a new model.
        """

        for field in model.__dict__["_modules"].values():
            if isinstance(field, nn.Linear):
                self.first_linear_layer = field
                break
            elif isinstance(field, nn.ModuleList):
                self.first_linear_layer = field[0]
                break

        self.lr = lr
        self.trace_idx += 1

        optim_name = type(optimizer).__name__
        self.train_loss_fig.data[self.trace_idx].name = f"{optim_name}, {lr}"
        self.grad_to_weight_fig.data[self.trace_idx].name = f"{optim_name}, {lr}"
        self.test_acc_fig.data[self.trace_idx].name = f"{optim_name}, {lr}"

    def plot_gradients_and_loss(self, batch_idx: int, loss: float) -> None:
        if batch_idx % self.log_interval == 0:
            self.train_loss_fig.data[self.trace_idx].y += (loss,)

            layer = self.first_linear_layer
            grad_to_weight_ratio = (
                self.lr * layer.weight.grad.std() / layer.weight.std()
            ).item()

            self.grad_to_weight_fig.data[self.trace_idx].y += (grad_to_weight_ratio,)

    def plot_accuracy(self, epoch: int, accuracy: float) -> None:
        self.test_acc_fig.data[self.trace_idx].x += (epoch,)
        self.test_acc_fig.data[self.trace_idx].y += (accuracy,)

In [18]:
def train_epoch(
    model: torch.nn.Module,
    device: torch.device,
    train_loader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    epoch: int,
    log_interval: int,
    visualizer: TrainingVisualizer,
    verbose: bool = False,
) -> None:
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move data to the device (e.g. GPU #0) where model parameters are.
        data, target = data.to(device), target.to(device)

        # Since `backward()` will accumuate gradients, remember to reset them.
        optimizer.zero_grad()

        # Forward pass.
        output = model(data)  # Logs of class probabilites, shape (B, 10)
        loss = F.nll_loss(output, target)  # Shape (1,)

        # Backward pass.
        loss.backward()

        visualizer.plot_gradients_and_loss(batch_idx, loss.item())

        # For each parameter, updates the tensor's value using its .grad.
        optimizer.step()

        if batch_idx % log_interval == 0:
            if verbose:
                done, total = batch_idx * len(data), len(train_loader.dataset)
                print(
                    f"Train Epoch: {epoch} [{done}/{total} images ({done / total:.0%})]\t"
                    + f"Loss: {loss.item():.6f}"
                )

In [19]:
def test(
    model: torch.nn.Module,
    device: torch.device,
    test_loader: torch.utils.data.DataLoader,
    epoch: int,
    visualizer: TrainingVisualizer,
    verbose: bool = False,
) -> None:
    model.eval()
    test_loss = 0
    n_correct = 0
    n_total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # Accumulate sum of dataitem losses and only divide by n_total at the end.
            test_loss += F.nll_loss(output, target, reduction="sum").item()
            pred = output.argmax(dim=1)
            n_correct += (pred == target).sum().item()
            n_total += len(target)

    test_loss /= n_total

    if verbose:
        print(f"\nTest loss: {test_loss:.4f}, Accuracy: {n_correct}/{n_total} ({n_correct/n_total:.0%})\n")
    visualizer.plot_accuracy(epoch, 100.0 * n_correct / n_total)

## Hyperparameters

In [20]:
# Training uses more memory than test (due to gradient computations),
# so we can set test_batch_size to be larger.
epochs = 5
batch_size = 256
test_batch_size = 1000
lr = 1e-2
log_interval = 10

# Use CUDA if a CUDA GPU is available, otherwise we'll run on CPU.
# You could also use "mps" for Apple Silicon, and some others,
# though support for less common operations may vary.
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Dataset and dataloader
Any sequence of dataitems (and even some iterables) can be used as a ***dataset***.<br>
A dataitem can be for example an (input image, output label) pair.<br>
Torch datasets usually allow passing a transformation: it is then applied whenever you retrieve an item with `dataset[i]`.

A ***dataloader*** manages parallel loading (it can create worker threads that call `dataset[i]` in parallel), batching, shuffling, sometimes moving data between devices.

In [21]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=(0.1307,), std=(0.3081,))]
)
train_dataset = datasets.MNIST("../data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST("../data", train=False, transform=transform)
len(train_dataset), len(test_dataset)

100%|██████████| 9.91M/9.91M [00:00<00:00, 20.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 493kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.56MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.68MB/s]


(60000, 10000)

In [22]:
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, **kwargs)

## Running it all

In [23]:
model = Net().to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)

visualizer = TrainingVisualizer(log_interval=log_interval)
visualizer.track_model(model, optimizer, lr)

for epoch in range(1, epochs + 1):
    train_epoch(
        model,
        device,
        train_loader,
        optimizer,
        epoch,
        log_interval,
        visualizer,
        verbose=True,
    )
    test(model, device, test_loader, epoch, visualizer, verbose=True)

FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '1cbe5e9b-4d82-4ae1-be22-54a5ead5292d',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '15fe1ea8-1072-4414-bad8-f6b4f10b9402',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '11a98d05-d61c-4722-8402-cd5e07bc8258',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f13f4326-464a-4ff9-93c8-df22b6d1295a',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '26fc8ff0-68ff-4c9c-9fa5-b7160ebd15e2',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'acc547b4-e967-499b-be54-1dd3f2ae8b53',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'cead9970-5041-413d-8523-052988c8e359',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '9a975236-1e87-493a-afb9-7b15c66586e5',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'fc5357cd-ac90-45c0-b2fc-f2dba22e2d07',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b9f57e60-5b9e-4b98-b4cd-7c487a339f8b',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '977cb09d-13c1-4bdf-a515-93ef4f87ec7c',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '3687ef26-27b5-426a-b63e-4ef9201ba2ff',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '2b52981b-53fd-47d3-8de9-c6169040b3c5',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b9b1f49b-26ac-4c56-adb0-4a272ff43a7f',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'd

Train Epoch: 1 [0/60000 images (0%)]	Loss: 2.314002
Train Epoch: 1 [2560/60000 images (4%)]	Loss: 0.542665
Train Epoch: 1 [5120/60000 images (9%)]	Loss: 0.285346
Train Epoch: 1 [7680/60000 images (13%)]	Loss: 0.294533
Train Epoch: 1 [10240/60000 images (17%)]	Loss: 0.258572
Train Epoch: 1 [12800/60000 images (21%)]	Loss: 0.301675
Train Epoch: 1 [15360/60000 images (26%)]	Loss: 0.362897
Train Epoch: 1 [17920/60000 images (30%)]	Loss: 0.251745
Train Epoch: 1 [20480/60000 images (34%)]	Loss: 0.151805
Train Epoch: 1 [23040/60000 images (38%)]	Loss: 0.172183
Train Epoch: 1 [25600/60000 images (43%)]	Loss: 0.143477
Train Epoch: 1 [28160/60000 images (47%)]	Loss: 0.272512
Train Epoch: 1 [30720/60000 images (51%)]	Loss: 0.205603
Train Epoch: 1 [33280/60000 images (55%)]	Loss: 0.225461
Train Epoch: 1 [35840/60000 images (60%)]	Loss: 0.167229
Train Epoch: 1 [38400/60000 images (64%)]	Loss: 0.131399
Train Epoch: 1 [40960/60000 images (68%)]	Loss: 0.119783
Train Epoch: 1 [43520/60000 images (73%)]

Dalsze rozwiązanie zadań
Dodanie SGD z Momentum

In [25]:
model2 = Net().to(device)

optimizer2 = optim.SGD(model2.parameters(), lr=lr, momentum=0.9)

visualizer = TrainingVisualizer(log_interval=log_interval)
visualizer.track_model(model2, optimizer2, lr)

for epoch in range(1, epochs + 1):
    train_epoch(
        model2,
        device,
        train_loader,
        optimizer2,
        epoch,
        log_interval,
        visualizer,
        verbose=True,
    )
    test(model2, device, test_loader, epoch, visualizer, verbose=True)


FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'c35c18f9-ca3b-4bc5-84de-b2f7bc1e4c13',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f6a71f67-217d-460d-bf50-5dc7d403ae0d',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'ba47b799-0dc5-4f54-b17c-7ba75768fa26',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '572cd3ad-8527-4ef5-b154-07f8c345e4a3',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '20d6a59b-18b8-47fc-94ed-755adb35c474',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b7345d85-75a1-4363-90a1-316823de77e0',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '78a53771-b6e2-45f5-88c2-95dc9766b1c1',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '67aea54f-377f-42cb-99a1-03a4b86d60b9',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '6d3aae45-3947-432f-9e6c-3ee3dd846ba9',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '20c61a70-dbec-4362-8441-d8ebb583f127',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'c9ace6dc-889f-4511-883f-678a0e4e3b22',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '31838dd0-bcad-4160-a478-163f00655086',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '5dafd6d2-5738-4698-a005-dc6433ba5db3',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '33b6a1ec-1bf3-423c-ad7d-0db816cf0428',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'd

Train Epoch: 1 [0/60000 images (0%)]	Loss: 2.321206
Train Epoch: 1 [2560/60000 images (4%)]	Loss: 2.236222
Train Epoch: 1 [5120/60000 images (9%)]	Loss: 1.960521
Train Epoch: 1 [7680/60000 images (13%)]	Loss: 1.499186
Train Epoch: 1 [10240/60000 images (17%)]	Loss: 0.923164
Train Epoch: 1 [12800/60000 images (21%)]	Loss: 0.643933
Train Epoch: 1 [15360/60000 images (26%)]	Loss: 0.561816
Train Epoch: 1 [17920/60000 images (30%)]	Loss: 0.438632
Train Epoch: 1 [20480/60000 images (34%)]	Loss: 0.340343
Train Epoch: 1 [23040/60000 images (38%)]	Loss: 0.323815
Train Epoch: 1 [25600/60000 images (43%)]	Loss: 0.401676
Train Epoch: 1 [28160/60000 images (47%)]	Loss: 0.321222
Train Epoch: 1 [30720/60000 images (51%)]	Loss: 0.422759
Train Epoch: 1 [33280/60000 images (55%)]	Loss: 0.329050
Train Epoch: 1 [35840/60000 images (60%)]	Loss: 0.337250
Train Epoch: 1 [38400/60000 images (64%)]	Loss: 0.293146
Train Epoch: 1 [40960/60000 images (68%)]	Loss: 0.237991
Train Epoch: 1 [43520/60000 images (73%)]

In [28]:
epochs = 3
log_interval = 10
learning_rates = [1e-2, 1e-3, 1e-4]

optimizers = {
    "Adam": lambda params, lr: torch.optim.Adam(params, lr=lr),
    "SGD+Momentum": lambda params, lr: torch.optim.SGD(params, lr=lr, momentum=0.9),
}

visualizers = []

for opt_name, opt_fn in optimizers.items():
    for lr in learning_rates:
        print(f"\n=== Trening: {opt_name} (lr={lr}) ===")

        model = Net().to(device)
        optimizer = opt_fn(model.parameters(), lr)

        visualizer = TrainingVisualizer(log_interval=log_interval)
        visualizer.track_model(model, optimizer, lr)

        for epoch in range(1, epochs + 1):
            train_epoch(
                model=model,
                device=device,
                train_loader=train_loader,
                optimizer=optimizer,
                epoch=epoch,
                log_interval=log_interval,
                visualizer=visualizer,
                verbose=True,
            )
            test(
                model=model,
                device=device,
                test_loader=test_loader,
                epoch=epoch,
                visualizer=visualizer,
                verbose=True,
            )

        visualizers.append(visualizer)



=== Trening: Adam (lr=0.01) ===


FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'eae732de-a31f-4ad0-b0b6-cc7f1183f59d',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '93835307-3828-4b31-ac4c-f96818105f52',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'ca1046e9-ddfb-41dc-9a4e-1b37ac07db68',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '323663bf-0767-47e7-8588-27a7aace7378',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'a99f8149-b734-4e2e-95ea-c54717f88287',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '7ceee541-e0f1-439e-a546-c2832600fae0',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '2637f8e5-84c5-4341-b46e-22e36db530e0',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'dc671f28-0e58-4e79-ac57-346e7f41011c',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'da9ddc58-84f5-4af2-b421-f445e4d5d942',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '46ca95a0-a9b8-4917-a030-109c521fdc45',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '3129e0a8-7596-4341-8829-9e47b3c2a8ea',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '010d5342-0b29-45e3-8350-14ea3ec7555e',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'ab4c1817-eec0-400c-8925-f34354e3c3f7',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '0c352894-ca6c-4cc0-bbc5-0c0e72b2dc9e',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '1

Train Epoch: 1 [0/60000 images (0%)]	Loss: 2.303091
Train Epoch: 1 [2560/60000 images (4%)]	Loss: 0.572823
Train Epoch: 1 [5120/60000 images (9%)]	Loss: 0.420186
Train Epoch: 1 [7680/60000 images (13%)]	Loss: 0.348531
Train Epoch: 1 [10240/60000 images (17%)]	Loss: 0.307689
Train Epoch: 1 [12800/60000 images (21%)]	Loss: 0.295208
Train Epoch: 1 [15360/60000 images (26%)]	Loss: 0.212880
Train Epoch: 1 [17920/60000 images (30%)]	Loss: 0.226469
Train Epoch: 1 [20480/60000 images (34%)]	Loss: 0.235072
Train Epoch: 1 [23040/60000 images (38%)]	Loss: 0.267371
Train Epoch: 1 [25600/60000 images (43%)]	Loss: 0.240025
Train Epoch: 1 [28160/60000 images (47%)]	Loss: 0.259230
Train Epoch: 1 [30720/60000 images (51%)]	Loss: 0.229618
Train Epoch: 1 [33280/60000 images (55%)]	Loss: 0.299561
Train Epoch: 1 [35840/60000 images (60%)]	Loss: 0.171680
Train Epoch: 1 [38400/60000 images (64%)]	Loss: 0.195212
Train Epoch: 1 [40960/60000 images (68%)]	Loss: 0.199816
Train Epoch: 1 [43520/60000 images (73%)]

...

Train Epoch: 1 [0/60000 images (0%)]	Loss: 2.319382
Train Epoch: 1 [2560/60000 images (4%)]	Loss: 2.315849
Train Epoch: 1 [5120/60000 images (9%)]	Loss: 2.279009
Train Epoch: 1 [7680/60000 images (13%)]	Loss: 2.257570
Train Epoch: 1 [10240/60000 images (17%)]	Loss: 2.243339
Train Epoch: 1 [12800/60000 images (21%)]	Loss: 2.221483
Train Epoch: 1 [15360/60000 images (26%)]	Loss: 2.176315
Train Epoch: 1 [17920/60000 images (30%)]	Loss: 2.139246
Train Epoch: 1 [20480/60000 images (34%)]	Loss: 2.108302
Train Epoch: 1 [23040/60000 images (38%)]	Loss: 2.079589
Train Epoch: 1 [25600/60000 images (43%)]	Loss: 2.021056
Train Epoch: 1 [28160/60000 images (47%)]	Loss: 1.981245
Train Epoch: 1 [30720/60000 images (51%)]	Loss: 1.904228
Train Epoch: 1 [33280/60000 images (55%)]	Loss: 1.855702
Train Epoch: 1 [35840/60000 images (60%)]	Loss: 1.732632
Train Epoch: 1 [38400/60000 images (64%)]	Loss: 1.666634
Train Epoch: 1 [40960/60000 images (68%)]	Loss: 1.608716
Train Epoch: 1 [43520/60000 images (73%)]

FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'fb77b49c-c902-40ad-bb3d-ddb3840f141a',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'c321931b-1753-4a57-9a85-1a17a97bf15d',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '74890098-a936-4952-9a63-14a02012f7fa',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'c0c24d2b-8e45-488b-973c-364eca207893',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b72b0bf1-4c6f-49c3-b0ff-cc90a7a3b81a',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'da861977-45e2-4fa1-b708-5045a411df07',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '4e935631-7ecf-4774-9653-03bf0b7dc51e',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b9b11101-af11-4378-9b5b-d1d0927a90e5',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '08c01eab-e2d9-437a-b990-580609c3ac1c',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b75a5dee-388e-4971-af67-b9a2714e5096',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e3138dfb-1bec-48b8-82bc-46f823aa280b',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b3842ba1-ba8e-4f0d-b093-288f4c92f19a',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '35a8514a-3e21-47c7-af37-25e491e29a0e',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '96762ce7-ab95-4e4e-89d3-794a10c4fe40',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '5

Train Epoch: 1 [0/60000 images (0%)]	Loss: 2.301081
Train Epoch: 1 [2560/60000 images (4%)]	Loss: 2.305673
Train Epoch: 1 [5120/60000 images (9%)]	Loss: 2.301512
Train Epoch: 1 [7680/60000 images (13%)]	Loss: 2.298600
Train Epoch: 1 [10240/60000 images (17%)]	Loss: 2.297861
Train Epoch: 1 [12800/60000 images (21%)]	Loss: 2.286270
Train Epoch: 1 [15360/60000 images (26%)]	Loss: 2.293312
Train Epoch: 1 [17920/60000 images (30%)]	Loss: 2.284836
Train Epoch: 1 [20480/60000 images (34%)]	Loss: 2.274211
Train Epoch: 1 [23040/60000 images (38%)]	Loss: 2.277310
Train Epoch: 1 [25600/60000 images (43%)]	Loss: 2.277219
Train Epoch: 1 [28160/60000 images (47%)]	Loss: 2.277612
Train Epoch: 1 [30720/60000 images (51%)]	Loss: 2.274408
Train Epoch: 1 [33280/60000 images (55%)]	Loss: 2.272651
Train Epoch: 1 [35840/60000 images (60%)]	Loss: 2.261276
Train Epoch: 1 [38400/60000 images (64%)]	Loss: 2.271854
Train Epoch: 1 [40960/60000 images (68%)]	Loss: 2.254645
Train Epoch: 1 [43520/60000 images (73%)]

In [29]:
class Net(nn.Module):
    def __init__(self, hidden_sizes=[128, 128], input_size=784, output_size=10):
        super(Net, self).__init__()

        layer_sizes = [input_size] + hidden_sizes + [output_size]

        self.layers = nn.ModuleList([
            nn.Linear(layer_sizes[i], layer_sizes[i + 1])
            for i in range(len(layer_sizes) - 1)
        ])

    def forward(self, x):
        x = x.view(x.size(0), -1)

        for layer in self.layers[:-1]:
            x = F.relu(layer(x))

        x = F.log_softmax(self.layers[-1](x), dim=1)
        return x


In [30]:
# Standardowa sieć 784–128–128–10
model = Net(hidden_sizes=[128, 128], input_size=784, output_size=10)

# Głębsza sieć
model_deep = Net(hidden_sizes=[256, 128, 64], input_size=784, output_size=10)

# Płytsza sieć
model_shallow = Net(hidden_sizes=[64], input_size=784, output_size=10)


In [31]:
lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
epochs = 5
log_interval = 10

visualizer = TrainingVisualizer(log_interval=log_interval)
visualizer.track_model(model, optimizer, lr)

for epoch in range(1, epochs + 1):
    train_epoch(
        model=model,
        device=device,
        train_loader=train_loader,
        optimizer=optimizer,
        epoch=epoch,
        log_interval=log_interval,
        visualizer=visualizer,
        verbose=True,
    )

    test(
        model=model,
        device=device,
        test_loader=test_loader,
        epoch=epoch,
        visualizer=visualizer,
        verbose=True,
    )


FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '325e159a-c240-4bfd-aa1a-aa736052e7a6',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '1f0bb8e9-c474-4aef-b7d2-05b49a9254f0',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '6fe6d604-f579-45ee-8c7c-eab050d9646b',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'c8d894ff-3d49-49ee-9ed6-a0809da4f8bf',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '84746b6c-1450-49cf-b817-6d114d9762a2',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e5463919-4372-4687-b172-1c48b14f6e80',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '73c38d78-53ac-40a9-b2f3-074115da4980',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '82a1e382-bb74-4857-80e0-650e37867e6c',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '197f5095-f656-4537-a4d0-ebba48c2eed5',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '007e12f8-5976-4bc8-9207-1919a36c00a9',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '5a61a222-98de-4454-9d9d-e86eb972a31d',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '2525a320-97c3-4207-a6bb-788ec37f071a',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'a1494abc-57bf-4f0c-afae-59771fcb56eb',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '6358033d-36f3-48ae-bfde-4fe2cac5690b',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '3

Train Epoch: 1 [0/60000 images (0%)]	Loss: 2.302764
Train Epoch: 1 [2560/60000 images (4%)]	Loss: 1.129988
Train Epoch: 1 [5120/60000 images (9%)]	Loss: 0.528307
Train Epoch: 1 [7680/60000 images (13%)]	Loss: 0.476482
Train Epoch: 1 [10240/60000 images (17%)]	Loss: 0.351290
Train Epoch: 1 [12800/60000 images (21%)]	Loss: 0.345675
Train Epoch: 1 [15360/60000 images (26%)]	Loss: 0.288671
Train Epoch: 1 [17920/60000 images (30%)]	Loss: 0.361863
Train Epoch: 1 [20480/60000 images (34%)]	Loss: 0.386275
Train Epoch: 1 [23040/60000 images (38%)]	Loss: 0.241892
Train Epoch: 1 [25600/60000 images (43%)]	Loss: 0.236820
Train Epoch: 1 [28160/60000 images (47%)]	Loss: 0.267111
Train Epoch: 1 [30720/60000 images (51%)]	Loss: 0.218191
Train Epoch: 1 [33280/60000 images (55%)]	Loss: 0.173341
Train Epoch: 1 [35840/60000 images (60%)]	Loss: 0.155599
Train Epoch: 1 [38400/60000 images (64%)]	Loss: 0.178446
Train Epoch: 1 [40960/60000 images (68%)]	Loss: 0.215218
Train Epoch: 1 [43520/60000 images (73%)]

In [32]:
lr = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.6)
epochs = 5
log_interval = 10

visualizer = TrainingVisualizer(log_interval=log_interval)
visualizer.track_model(model, optimizer, lr)

for epoch in range(1, epochs + 1):
    train_epoch(
        model=model,
        device=device,
        train_loader=train_loader,
        optimizer=optimizer,
        epoch=epoch,
        log_interval=log_interval,
        visualizer=visualizer,
        verbose=True,
    )

    test(
        model=model,
        device=device,
        test_loader=test_loader,
        epoch=epoch,
        visualizer=visualizer,
        verbose=True,
    )


FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '0ecd7927-08b1-40ab-b234-083c922ff3cc',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'dc2e62ed-84d9-452e-aeab-fada0b4673e6',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '7279cd9e-ecfb-4003-82ae-97bdb81dcccd',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '9d3d12e4-7509-490e-828f-ba4cfc415cdc',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '4065573c-620d-439d-88a1-b9e7985225e6',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '51693ac1-224a-4a3d-a71e-bca313d11edf',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f4851cf9-1cd5-4738-8e8a-21505fcf482e',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b0fa032e-9202-4e86-b613-acb5b3773346',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '60634d81-38e1-4b96-bec6-9986914a1fda',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '32cdd1d9-d788-49fe-986c-59651eec6000',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'bd4e96a2-ba18-4df9-bf5c-73a7ce23973e',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'c93cc243-d38a-4b1f-91f9-2eb022b0a7be',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f0e4b10c-24fa-4dae-b941-c8752db5fc08',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '28809194-8f7b-4854-a874-c7fdd0825721',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b

Train Epoch: 1 [0/60000 images (0%)]	Loss: 0.066241
Train Epoch: 1 [2560/60000 images (4%)]	Loss: 0.042779
Train Epoch: 1 [5120/60000 images (9%)]	Loss: 0.047336
Train Epoch: 1 [7680/60000 images (13%)]	Loss: 0.036640
Train Epoch: 1 [10240/60000 images (17%)]	Loss: 0.046157
Train Epoch: 1 [12800/60000 images (21%)]	Loss: 0.026635
Train Epoch: 1 [15360/60000 images (26%)]	Loss: 0.054359
Train Epoch: 1 [17920/60000 images (30%)]	Loss: 0.049450
Train Epoch: 1 [20480/60000 images (34%)]	Loss: 0.019412
Train Epoch: 1 [23040/60000 images (38%)]	Loss: 0.058625
Train Epoch: 1 [25600/60000 images (43%)]	Loss: 0.022634
Train Epoch: 1 [28160/60000 images (47%)]	Loss: 0.055872
Train Epoch: 1 [30720/60000 images (51%)]	Loss: 0.035755
Train Epoch: 1 [33280/60000 images (55%)]	Loss: 0.064375
Train Epoch: 1 [35840/60000 images (60%)]	Loss: 0.050264
Train Epoch: 1 [38400/60000 images (64%)]	Loss: 0.035593
Train Epoch: 1 [40960/60000 images (68%)]	Loss: 0.061628
Train Epoch: 1 [43520/60000 images (73%)]

Warte zauważenia, że SGD z momentum osiągnął 98% już w pierwszej epoce